# 1. Setup Mario

In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [2]:
# importing the game from gym
import gym_super_mario_bros
# next was to import Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Followed by SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
# Setting up the game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# 2. Preprocess Environment

In [4]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [5]:
# Installation of stable baselines for Reinforced Learning
!pip install stable-baselines3[extra]

In [6]:
# Importing Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib for visualizing
from matplotlib import pyplot as plt

# 3. Train the RL Model

In [7]:
# Importing os for management of file path
import os 
# Import PPO, DQN, Gail as RL learning agents
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3 import A2C
# Import Base Callback to save models
from stable_baselines3.common.callbacks import BaseCallback

In [8]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, freq_check, store_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.freq_check = freq_check
        self.store_path = store_path

    def _init_callback(self):
        if self.store_path is not None:
            os.makedirs(self.store_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.freq_check == 0:
            model_path = os.path.join(self.store_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [9]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [10]:
# Setup model saving callback
callback = TrainAndLoggingCallback(freq_check=10000, store_path=CHECKPOINT_DIR)

In [11]:
# This is the AI model started
model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, seed = 50, buffer_size=100
            ) 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [12]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=900000, callback=callback)

Logging to ./logs/DQN_3


C:\ProgramData\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.42e+04 |
|    ep_rew_mean      | 612      |
|    exploration_rate | 0.399    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 438      |
|    time_elapsed     | 130      |
|    total_timesteps  | 56980    |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.427    |
|    n_updates        | 1744     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.5e+03  |
|    ep_rew_mean      | 1.5e+03  |
|    exploration_rate | 0.198    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 269      |
|    time_elapsed     | 282      |
|    total_timesteps  | 75961    |
| train/              |          |
|    learning_rate    | 1e-06    |
|    loss             | 0.684    |
|    n_updates      

In [13]:
model.save('thisisdqntestresult')

In [14]:
# Load the model
model = DQN.load('./train/best_model_900000')

In [15]:


state = env.reset()

